In [1]:
import requests
import time
from tqdm import tqdm  # For the progress bar
import pandas as pd   # Import pandas for DataFrame
import os  
import requests
import time
from tqdm import tqdm  # For the progress bar
import pandas as pd   # Import pandas for DataFrame
import os  

#
#with open('api_key.txt', 'r') as file:
#    API_KEY = file.read().strip() 
#print(API_KEY)
API_KEY = "P4WFtQQ8zMc89xCESY0x2E183bfOpHRjvY159e4H3HcOJ6GNaDI=--I3/cG6WtrxR9ac4t--/e+gBW1nRPXJS7PggzSspg=="
BASE_URL = 'https://api.nexusmods.com/v1/'
headers = {
    'apikey': API_KEY,
    'Accept': 'application/json'
}
OUTPUT_CSV = r'D:\Repos\nexus_mods_datascrape\nexus_mods_data.csv'


## Getting all games on Nexus to select

In [2]:
def get_all_games():
    """Retrieve a list of all games supported by Nexus Mods."""
    url = f"{BASE_URL}games"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        games = response.json()
        print(f"Total games retrieved: {len(games)}")
        return games
    else:
        print(f"Error {response.status_code}: Unable to fetch games list.")
        return []


In [14]:
GAME_TITLE_TO_DOMAIN = {
    'skyrim special edition': 'skyrimspecialedition',
    'skyrim': 'skyrim',
    'fallout 4': 'fallout4',
    'fallout new vegas': 'newvegas',
    'cyberpunk 2077': 'cyberpunk2077',
    'stardew valley': 'stardewvalley',
    'oblivion': 'oblivion',
    'baldurs gate 3': 'baldursgate3',
    'fallout 3': 'fallout3',
    'the witcher 3': 'witcher3',
    'mount and blade 2': 'mountandblade2bannerlord',
    'monster hunter': 'monsterhunterworld',
    'blade and sorcery': 'bladeandsorcery',
    'dragon age origins': 'dragonage',
    'morrowind': 'morrowind',
    'modding tools': 'moddingtools',
    'starfield': 'starfield',
    'elden ring': 'eldenring'
}


In [15]:
def get_latest_mod_id(game_domain):
    """Retrieve the latest mod ID for a given game domain."""
    url = f"{BASE_URL}games/{game_domain}/mods/latest_added.json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        mods = response.json()
        if mods and isinstance(mods, list):
            latest_mod = mods[0]  # Assuming the first mod is the latest
            latest_mod_id = latest_mod.get('mod_id')
            return latest_mod_id
        else:
            print(f"No mods found for game domain {game_domain}.")
            return None
    else:
        print(f"Error {response.status_code}: Unable to fetch latest mod for {game_domain}.")
        return None

In [19]:
def get_mods_with_categories(game_domain, max_mods=None, batch_size=100):
    """Retrieve mods for a game and collect their details in a DataFrame."""
    latest_mod_id = get_latest_mod_id(game_domain)
    if not latest_mod_id:
        return pd.DataFrame()  # If no mods available, return an empty DataFrame

    if max_mods is not None:
        latest_mod_id = min(latest_mod_id, max_mods)
    mod_data_list = []
    mods_processed = 0
    total_mods = latest_mod_id
    print(f"Processing {total_mods} mods for game '{game_domain}'")
    file_exists = os.path.isfile(OUTPUT_CSV)

    for mod_id in tqdm(range(1, latest_mod_id + 1), desc=f"Processing {game_domain}", unit="mod"):
        while True:
            url = f"{BASE_URL}games/{game_domain}/mods/{mod_id}.json"
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                mod = response.json()
                mod_data = {
                'mod_id': mod.get('mod_id'),
                'name': mod.get('name'),
                'game_domain': game_domain,
                'categoryid': mod.get('category_id'),
                'category_name': mod.get('category_name'),
                'author': mod.get('author'),
                'uploaded_by': mod.get('uploaded_by'),
                'user_id': mod.get('user', {}).get('member_id'),
                'user_name': mod.get('user', {}).get('name'),
                'user_group_id': mod.get('user', {}).get('member_group_id'),
                'created_time': mod.get('created_time'),
                'updated_time': mod.get('updated_time'),
                'downloads': mod.get('mod_unique_downloads'),
                'unique_downloads': mod.get('mod_downloads'),
                'endorsement_count': mod.get('endorsement_count'),
                'summary':mod.get('summary'),
                'description':mod.get('description'),
                
                'contains_adult_content': mod.get('contains_adult_content'),
                'status': mod.get('status')
            }
                mod_data_list.append(mod_data)
                mods_processed += 1

                # Save data in batches
                if len(mod_data_list) >= batch_size:
                    df_batch = pd.DataFrame(mod_data_list)
                    df_batch.to_csv(OUTPUT_CSV, mode='a', index=False, header=not file_exists)
                    file_exists = True  # Once header is written once, set this to True
                    mod_data_list.clear()  # Clear the list after saving

                break  # Exit the while loop if successful
            elif response.status_code == 429:  # Rate limit exceeded
                retry_after = int(response.headers.get('Retry-After', 600))  
                print(f"Rate limit exceeded. Retrying after {retry_after} seconds.")
                time.sleep(retry_after)  # Wait for the specified time before retrying
            else:
                print(f"Error {response.status_code}: Unable to fetch mod {mod_id} for {game_domain}.")
                break  # Exit the while loop on other error codes

    if mod_data_list:
        # Final save for any remaining data
        df_batch = pd.DataFrame(mod_data_list)
        df_batch.to_csv(OUTPUT_CSV, mode='a', index=False, header=not file_exists)
        mod_data_list.clear()

    print(f"\nTotal mods processed for game '{game_domain}': {mods_processed}")
    print(f"Data saved incrementally to '{OUTPUT_CSV}'.")

    df_mods = pd.read_csv(OUTPUT_CSV)
    return df_mods

In [20]:
def main():
    if os.path.isfile(OUTPUT_CSV):
        os.remove(OUTPUT_CSV)
        print(f"Existing '{OUTPUT_CSV}' file removed.")

    # Use only the specified games
    selected_games = [
        #'skyrim special edition',
        #'skyrim',
        #'fallout 4',
        #'fallout new vegas',
        #'cyberpunk 2077',
        'stardew valley',
        #'oblivion',
        'baldurs gate 3',
        #'fallout 3',
        #'the witcher 3',
        #'mount and blade 2',
        #'monster hunter',
        #'blade and sorcery',
        'dragon age origins',
        #'morrowind',
        #'modding tools',
        'starfield',
        'elden ring'
    ]

    game_domains = []
    for game_title in selected_games:
        domain = GAME_TITLE_TO_DOMAIN.get(game_title.lower())
        if domain:
            game_domains.append(domain)
        else:
            print(f"Domain name not found for game '{game_title}'. Please check the mapping.")

    all_mods_df_list = []

    for game_domain in game_domains:
        print(f"\nProcessing game domain: {game_domain}")
        df_mods = get_mods_with_categories(game_domain, batch_size=50)
        if not df_mods.empty:
            all_mods_df_list.append(df_mods)

    if all_mods_df_list:
        all_mods_df = pd.concat(all_mods_df_list, ignore_index=True)
        print("\nCombined DataFrame shape:", all_mods_df.shape)
    else:
        all_mods_df = pd.DataFrame()
        print("\nNo mod data collected.")

    return all_mods_df  # Return the combined DataFrame


In [22]:

if __name__ == "__main__":
    all_mods_df = main()
    # Now you can use all_mods_df for data analysis
    # Example:
    print("\nFirst few rows of the DataFrame:")
    print(all_mods_df.head())


Processing game domain: stardewvalley
Processing 28137 mods for game 'stardewvalley'


Processing stardewvalley:   0%|                                                  | 12/28137 [00:04<3:16:29,  2.39mod/s]

Error 404: Unable to fetch mod 12 for stardewvalley.


Processing stardewvalley:   0%|                                                  | 49/28137 [00:34<3:04:52,  2.53mod/s]

Error 404: Unable to fetch mod 49 for stardewvalley.


...

Error 404: Unable to fetch mod 992 for stardewvalley.


Processing stardewvalley:   4%|█▋                                               | 999/28137 [57:59<3:10:09,  2.38mod/s]

Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.
Rate limit exceeded. Retrying after 300 seconds.


Processing stardewvalley:   4%|█▋                                            | 999/28137 [1:48:03<48:55:15,  6.49s/mod]


KeyboardInterrupt: 